In [1]:
import cv2
import numpy as np
import matplotlib as plt
import pathlib
from os import listdir
from os.path import isfile, join

In [2]:
# define location of dataset and return all files
dataset_location = "example_data/_input_multi/"
target_dir = "example_data/COCO/"
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]

# next, sort files into images, depth maps, segmentation maps, data, and colony info
dataset_img = []
dataset_depth = []
dataset_seg = []
dataset_data = []
dataset_colony = dataset_location + "/ColonieInfo.csv"

for file in all_files:
    loc = dataset_location + "/" + file
    if file[-7:-4] == "Img":
        dataset_img.append(loc)
    elif file[-7:-4] == "Seg":
        dataset_seg.append(loc)
    elif file[-9:-4] == "Depth":
        dataset_depth.append(loc)
    elif file[-8:-4] == "Data":
        dataset_data.append(loc)
        
print("Found",len(all_files),"files...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# one entry for each successive ID is read
from csv import reader

colony = {'seed': 0,
            'ID': [],
         'scale': [],
        'weight': []}

with open(dataset_colony, 'r') as colony_file:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(colony_file)
        # iterate over each row in the csv using reader object
        for r, row in enumerate(csv_reader):
            if r == 0:
                colony['seed'] = row[0].split("=")[-1]
            else:
                colony['ID'].append(row[0].split("=")[-1])
                colony['weight'].append(row[1].split("_")[1] + "-" + row[1].split("_")[2])
                colony['scale'].append(float(row[2].split("=")[-1]))

print("Loaded colony file with seed",colony['seed'],"and",len(colony['ID']),"individuals.")

# get provided classes to create a dictionary of class IDs and class names
subject_class_names = np.unique(np.array(colony["weight"]))
subject_classes = {}
for id,sbj in enumerate(subject_class_names):
    subject_classes[str(sbj)] = id

print("\nA total of",len(subject_class_names),"unique classes have been found.")
print("The classes and respective class IDs are:\n",subject_classes)

Found 41 files...
reading ColonieInfo.csv
Loaded colony file with seed  123 and 10 individuals.

A total of 4 unique classes have been found.
The classes and respective class IDs are:
 {'00011-atta': 0, '00262-atta': 1, '00501-atta': 2, 'Sungaya-inexpectata': 3}


Now that we have the cleaned colony info, we can start loading the data associated with each frame.
For simplicity we will simply this a list of list as the number of individuals.

We will therefore access "data" as [frame] [individual] [attribute], where attributes will include [ID,bbox_x_0,bbox_y_0,...]

for now training and evaluating detectors, only the bounding box (and ID) info will be relevant

In [3]:
data = []

for file in dataset_data:
    # store all returned coordinates for each individual
    coords = []
        
    # open file in read mode
    with open(file, 'r') as read_obj:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # iterate over each row in the csv using reader object
        for row in csv_reader:
            # exclude camera projection row
            if not row[0].split(".")[0] == "camera_projection:":
                individual = [float(row[0].split(".")[0])]
                # row variable is a list that represents a row in csv
                for elem in row:
                    try:
                        individual.append(float(elem.split("=")[-1]))
                    except ValueError:
                        pass
                coords.append(individual)
        
    data.append(coords)
    
print("\nThe dataset has a total of", len(data),"generated frames.")

reading example_data/_input_multi//10_Data.csv
reading example_data/_input_multi//1_Data.csv
reading example_data/_input_multi//2_Data.csv
reading example_data/_input_multi//3_Data.csv
reading example_data/_input_multi//4_Data.csv
reading example_data/_input_multi//5_Data.csv
reading example_data/_input_multi//6_Data.csv
reading example_data/_input_multi//7_Data.csv
reading example_data/_input_multi//8_Data.csv
reading example_data/_input_multi//9_Data.csv

The dataset has a total of 10 generated frames.


As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [4]:
# first open and read the first line from the first imported data file
labels = []
with open(dataset_data[0], 'r') as read_obj:
    print("reading", file)
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    row_0 = next(csv_reader)  # gets the first line
    # iterate over each row in the csv using reader object
    for elem in row_0:
        try:
            labels.append((elem.split("=")[0].split("Bone.")[-1]))
        except ValueError:
            pass

# now let's define which labels NOT to use (in our case, all labels relating to wings)
# ... so that just means "omit all lables that start with 'w'"

matched_labels = []# [i for i, item in enumerate(labels) if item in omit_labels]
for l, label in enumerate(labels):
    if label[0] == "w":
        matched_labels.append(l-1)
        
print("\nCorresponding to the following indices:",matched_labels)

# show all used labels:
print("\nAll labels:")
print(labels)

reading example_data/_input_multi//9_Data.csv

Corresponding to the following indices: [143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]

All labels:
['0.BoundingBox.BoundMin.X', 'BoundingBox.BoundMin.Y', 'BoundingBox.BoundMax.X', 'BoundingBox.BoundMax.Y', 'b_t.X', 'b_t.Y', 'b_t.X_world', 'b_t.Y_world', 'b_t.Z_world', 'b_a_1.X', 'b_a_1.Y', 'b_a_1.X_world', 'b_a_1.Y_world', 'b_a_1.Z_world', 'b_a_2.X', 'b_a_2.Y', 'b_a_2.X_world', 'b_a_2.Y_world', 'b_a_2.Z_world', 'b_a_3.X', 'b_a_3.Y', 'b_a_3.X_world', 'b_a_3.Y_world', 'b_a_3.Z_world', 'b_a_4.X', 'b_a_4.Y', 'b_a_4.X_world', 'b_a_4.Y_world', 'b_a_4.Z_world', 'b_a_5.X', 'b_a_5.Y', 'b_a_5.X_world', 'b_a_5.Y_world', 'b_a_5.Z_world', 'b_a_5_end.X', 'b_a_5_end.Y', 'b_a_5_end.X_world', 'b_a_5_end.Y_world', 'b_a_5_end.Z_world', 'l_1_co_r.X', 'l_1_co_r.Y', 'l_1_co_r.X_world', 'l_1_co_r.Y_world', 'l_

Now that we have loaded data and colony info we can start plotting bounding boxes on top of their respective images

In [5]:
# transform between sRGB and linear colour space (optional)

def to_linear(srgb):
    linear = np.float32(srgb) / 255.0
    less = linear <= 0.04045
    linear[less] = linear[less] / 12.92
    linear[~less] = np.power((linear[~less] + 0.055) / 1.055, 2.4)
    return linear * 255.0

    
def from_linear(linear):
    srgb = linear.copy()
    less = linear <= 0.0031308
    srgb[less] = linear[less] * 12.92
    srgb[~less] = 1.055 * np.power(linear[~less], 1.0 / 2.4) - 0.055
    return srgb * 255.0

In [6]:
import json
# let's create a big dictionary to store all our dataset info and
# then dump it into a sexy COCO-conform json file
# based on the documentation : https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch
coco_data = {}

from datetime import datetime
date = datetime.today().strftime('%d.%m.%Y')

# edit any "info", "license", or "category" data here:
coco_data["info"] = {
        "description": "COCO_Style_FARTS_example_dataset",
        "url": "https://evo-biomech.ic.ac.uk/",
        "version": "1.0",
        "year": datetime.today().year,
        "contributor": "Fabian Plum, Rene Bulla, David Labonte",
        "date_created": date}

coco_data["licenses"] = [
        {
            "url": "http://creativecommons.org/licenses/by/4.0/",
            "id": 1,
            "name": "Attribution License"
        }
    ]

coco_data["categories"] = []

for s,sbj in enumerate(subject_class_names):
    coco_data["categories"].append(
            {
                "supercategory": "insect",
                "id": s + 1,
                "name": sbj,
                "keypoints":['b_t', 'b_a_1', 'b_a_2', 'b_a_3',
                             'b_a_4', 'b_a_5', 'b_a_5_end', 'l_1_co_r',
                             'l_1_tr_r', 'l_1_fe_r',  'l_1_ti_r', 'l_1_ta_r', 
                             'l_1_pt_r', 'l_1_pt_r_end', 'l_2_co_r', 'l_2_tr_r', 
                             'l_2_fe_r', 'l_2_ti_r', 'l_2_ta_r', 'l_2_pt_r', 
                             'l_2_pt_r_end', 'l_3_co_r', 'l_3_tr_r', 'l_3_fe_r', 
                             'l_3_ti_r', 'l_3_ta_r', 'l_3_pt_r', 'l_3_pt_r_end',
                             'w_1_r', 'w_1_r_end',  'w_2_r', 'w_2_r_end',
                             'l_1_co_l', 'l_1_tr_l', 'l_1_fe_l', 'l_1_ti_l',
                             'l_1_ta_l', 'l_1_pt_l', 'l_1_pt_l_end', 'l_2_co_l', 
                             'l_2_tr_l', 'l_2_fe_l', 'l_2_ti_l', 'l_2_ta_l',
                             'l_2_pt_l', 'l_2_pt_l_end', 'l_3_co_l', 'l_3_tr_l',
                             'l_3_fe_l', 'l_3_ti_l', 'l_3_ta_l', 'l_3_pt_l',
                             'l_3_pt_l_end', 'w_1_l', 'w_1_l_end', 'w_2_l',
                             'w_2_l_end', 'b_h', 'ma_r', 'ma_r_end',
                             'an_1_r', 'an_2_r', 'an_3_r', 'an_3_r_end',
                             'ma_l', 'ma_l_end', 'an_1_l', 'an_2_l', 
                             'an_3_l', 'an_3_l_end'],
                "skeleton":[
                    [2,1],[3,2],[4,3],
                    [5,4],[6,5],[7,6],
                    [9,8],[10,9],[11,10],[12,11],
                    [13,12],[14,13],[16,15],
                    [17,16],[18,17],[19,18],[20,19],
                    [21,20],[23,22],[24,23],
                    [25,24],[26,25],[27,26],[28,27],
                    [30,29],[32,31],
                    [34,33],[35,34],[36,35],
                    [37,36],[38,37],[39,38],
                    [41,40],[42,41],[43,42],[44,43],
                    [45,44],[46,45],[48,47],
                    [49,48],[50,49],[51,50],[52,51],
                    [53,52],[55,54],
                    [57,56],[58,1],[59,58],[60,59],
                    [61,58],[62,61],[63,62],[64,63],
                    [65,58],[66,65],[67,58],[68,67],
                    [69,68],[70,69]
                ]
            }
        )

# when adding images in the next step the following info needs to be given:
coco_data["images"] = []

# FORMATTING NOTES ["images"]

"""
"images": [
    {
        "id": ###### (-> generated ID, use i, needs to the same for annoations),
        "license": 1,
        "width": display_img.shape[0],
        "height": display_img.shape[0],
        "file_name": img.split('/')[-1][:-4] + "_synth" + ".JPG"
    },
    ...
"""

# FORMATTING NOTES ["annotations"]

"""
"annotations": [
    {
        "segmentation": [[x0,y0,x1,y1...xn,yn][x_0,y_0,...x_n,y_n]] (-> coordinates of mask outline, if seperated, multiple arrays can be passed),
        "area": #### (-> = to the sum of pixels inside the mask),
        "iscrowd": 0 (as we treat all individuals seperately),
        "image_id": # (-> = i when iterating over all images),
        "bbox": [bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]] (-> unlike darknet the original (sub-)pixel values are used here),
        "category_id": 1 (-> for now there is only one category, replace with class ID for multi class),
        "id": ##### (-> separate counter to i and im)
    },
"""

# each individual in the dataset is treated as a sparate annotation with a corresponding image ID
coco_data["annotations"] = []

In [7]:
# create unique colours for each ID
import numpy as np
import time

# alright. Let's take it from the top and fucking multi-thread this.
import threading
import queue
import sys
import os

def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class exportThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag_export:
        queueLock.acquire()
        if not workQueue_export.empty():
            
            data_input = q.get()
            i = data_input[0]
            img = data_input[1]
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_orig = display_img.copy()
            
            # only add images that contain visibile individuals
            is_empty = True
            
            img_name = target_dir + "/data/" + img.split('/')[-1][:-4] + "_synth" + ".jpg"

            img_info = []
                
            # compute visibility for each individual
            seg_img = cv2.imread(dataset_seg[i])
            seg_img_display = seg_img.copy()

            for im, individual in enumerate(data[i]):

                fontColor = (int(ID_colours[int(individual[0]),0]),
                             int(ID_colours[int(individual[0]),1]),
                             int(ID_colours[int(individual[0]),2]))
                bbox = fix_bounding_boxes(individual[1:5],max_val=display_img.shape)

                # FOR SOME REASON OCCASIONALLY THE ID OF THE SEG FILE IS LOWER THAN THE DATA FILE
                # with: ID = red_channel/255 * im
                # red_channel = (ID/im) * 255
                ID_red_val = int((individual[0]/len(colony['ID']))*255)
                
                contours_lowpoly = []
                
                try:
                    ID_mask = cv2.inRange(seg_img[bbox[1]:bbox[3],bbox[0]:bbox[2]], np.array([0,0, ID_red_val - 2]), np.array([0,0, ID_red_val + 2]))
                    indivual_occupancy = cv2.countNonZero(ID_mask)
                    
                    # the kernel size for both dilation and median blur are to be determined by the bbounding boxes relative size
                    rel_size = ((bbox[2] - bbox[0]) / display_img.shape[0] + (bbox[3] - bbox[1]) / display_img.shape[0]) / 2
                    # values range from 0 (tiny) to 1 (huge)
                    # required smoothing 5 to 95
                    rel_size_root = int(round((15 * rel_size)/2.)*2 + 1) # round to next odd integer
                    #print("img:", i, "individual:", im, "rel_size", rel_size, rel_size_root)

                    # to simplify the generated masks and counter compression artifacts the original mask is dilated
                    # https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html
                    kernel = np.ones((rel_size_root, rel_size_root), 'uint8')
                    ID_mask_dilated = cv2.dilate(ID_mask, kernel, iterations=1)
                    # use median blur to further smooth the edges of the binary mask
                    ID_mask_dilated = cv2.medianBlur(ID_mask_dilated,rel_size_root)

                    # pad segmentation subwindow to prevent contours from being cut off
                    """
                    pad_width = 20
                    ID_mask_dilated_padded = np.zeros([ID_mask_dilated.shape[0] + pad_width * 2 , ID_mask_dilated.shape[1] + pad_width * 2], 'uint8')
                    ID_mask_dilated_padded[pad_width:-pad_width,pad_width:-pad_width] = ID_mask_dilated
                    """

                    # find contours using cv2.CHAIN_APPROX_SIMPLE to minimise the number of control points
                    # use cv2.RETR_EXTERNAL instead of cv2.RETR_TREE to only return the outer most contours
                    # depending on the version of openCV the function findContours additionally returns the image
                    try:
                        contours, hierarchy = cv2.findContours(ID_mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
                    except:
                        useless_img, contours, hierarchy = cv2.findContours(ID_mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
                    # now sort contours by area and only keep the 4 largest parts 
                    contours = sorted(contours, key=cv2.contourArea, reverse=True)
                    if len(contours) > 3:
                        contours = contours[:4]

                    # finally we simplify the generated contours to decrease memory usage
                    # and fascilitate correct processing, using polygon approximation
                    for contour in contours:
                        # decrease epsilon for finer contours
                        contours_lowpoly.append(cv2.approxPolyDP(contour, epsilon=1, closed=True))


                    if len(threadList_export) == 1:
                        print("\nindividual",im,ID_mask_dilated.dtype)
                        print(hierarchy)
                        # draw the contours on the empty image
                        seg_img_display = seg_img.copy()
                        cv2.imshow("mask: ", ID_mask_dilated)
                        cv2.drawContours(seg_img_display[bbox[1]:bbox[3],bbox[0]:bbox[2]], contours, -1, (255,0,0), 3)
                        cv2.imshow("segmentation: ", seg_img_display[bbox[1]:bbox[3],bbox[0]:bbox[2]])
                        cv2.waitKey(1)
                        
                except:
                    indivual_occupancy = 1
                
                #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                bbox_occupancy = indivual_occupancy / bbox_area
                #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)
                
                class_ID = subject_classes[colony['weight'][int(individual[0])]] # here we use a single class, otherwise this can be replaced by size / scale values
                
                #cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                if bbox_occupancy > visibility_threshold:
                    #cv2.rectangle(display_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), fontColor, 2)
                    
                    # collect all joint info and convert into COCO readable format
                    # "keypoints" are arrays of length 3K, K is the total number of key points defined for a class 
                    # [x, y, v] with the key point visibility v:

                    # v=0   Indicates that this key point is not marked (in this case x=y=v=0）
                    # v=1   Indicates that this key point is marked but not visible(Obscured)
                    # v=2   Indicates that this key point is marked and visible at the same time
                    
                    # let's binarise the image and dilate it to make sure all points that are visible are found
                    seg_bin = cv2.inRange(seg_img, np.array([0,0, ID_red_val - 2]), np.array([0,245, ID_red_val + 2]))
                    kernel = np.ones((9,9),np.uint8)
                    seg_bin_dilated = cv2.dilate(seg_bin,kernel,iterations = 1)
        
                    
                    keypoints = []
                    img_shape = display_img.shape
                    for point in range(int(len(individual[5:])/5)):  
                        # check if point is located within the image
                        if point*5 + 4 in matched_labels or individual[point*5 + 5] > img_shape[0] or individual[point*5 + 5] < 0 or individual[point*5 + 6] > img_shape[1] or individual[point*5 + 6] < 0:
                            keypoints.extend([0,0,0]) # x=y=v=0 -> ignore keypoint
                        else:
                            # if it is, check its visibility
                            if seg_bin_dilated[int(individual[6 + point*5]),int(individual[5 + point*5])] == 255:                   
                                visibility_pt = 2 # point is visible
                            else:
                                visibility_pt = 1 # point is marked but obstructed
                       
                            keypoints.extend([int(individual[point*5 + 5]),int(individual[point*5 + 6]),visibility_pt])
                        #cv2.circle(display_img, (int(individual[point*2 + 5]),int(individual[point*2 + 6])), radius=3, color=fontColor, thickness=-1)
                        # let's see of this is really the centre
                        
                            
                            
                    if generate_dataset:
                        # now we need to convert all the info into the desired format.
                        segmentation_mask= []
                        
                        new_bbox = [display_img.shape[0],display_img.shape[1],0,0]
                        mask_area = 0
                        
                        if len(contours_lowpoly) != 0:
                            for contour in contours_lowpoly:
                                mask_area += cv2.contourArea(contour)
                                sub_mask = []
                                for coords in contour:
                                    sub_mask_x = int(bbox[0] + coords[0,0])
                                    sub_mask_y = int(bbox[1] + coords[0,1])
                                    sub_mask.append(sub_mask_x)
                                    sub_mask.append(sub_mask_y)

                                    if sub_mask_x < new_bbox[0]:
                                        new_bbox[0] = sub_mask_x
                                    if sub_mask_x > new_bbox[2]:
                                        new_bbox[2] = sub_mask_x

                                    if sub_mask_y < new_bbox[1]:
                                        new_bbox[1] = sub_mask_y
                                    if sub_mask_y > new_bbox[3]:
                                        new_bbox[3] = sub_mask_y

                                if len(sub_mask) >= 8:
                                    # only include polygons with at least 4 vertices
                                    segmentation_mask.append(sub_mask)
                                    is_empty = False

                        # now that we have a clean segmentation mask, we can refine the bounding box as well
                        
                        if not is_empty:
                            coco_data["annotations"].append({
                                    "segmentation": segmentation_mask, # (-> coordinates of mask outline, if seperated, multiple arrays can be passed),
                                    "area": mask_area, # (-> = to the sum of pixels inside the mask),
                                    "iscrowd": 0, #(as we treat all individuals seperately),
                                    "image_id":  i, # (-> = i when iterating over all images),
                                    "bbox": [new_bbox[0], new_bbox[1], new_bbox[2]-new_bbox[0], new_bbox[3]-new_bbox[1]], # (-> unlike darknet the original (sub-)pixel values are used here),
                                    "category_id": class_ID + 1, # (-> for now there is only one category),
                                    "id": int(str(i) + "000" + str(im)), # (-> joining i and im)
                                    "keypoints": keypoints
                                    })

                else:
                    pass
                    # create mask to highlight low visibility animals
                    #blk = np.zeros(display_img.shape, np.uint8)
                    #cv2.rectangle(blk, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), cv2.FILLED)

                    # display original bounding box
                    #cv2.rectangle(display_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), fontColor, 2)
                    # add text to discarded ID
                    #cv2.putText(display_img, "OCCLUDED", (bbox[0] + 10,bbox[3] - 35), font, fontScale, (0,0,255), lineType)

                    # blend the mask with original image
                    #display_img = cv2.addWeighted(display_img, 1.0, blk, 0.25, 1)

                    #print("Individual", int(individual[0]), "has been discarded due to excessive occlusion.")
                    #print("expected:",int((individual[0]/len(colony['ID']))*255))
                    
                            
            
            # uncomment to show resulting bounding boxes and masks
            if len(threadList_export) == 1:
                cv2.imshow("segmentation: " ,cv2.resize(seg_img_display, (int(seg_img.shape[1] / 2), 
                                                                  int(seg_img.shape[0] / 2))))
                cv2.imshow("labeled image", cv2.resize(display_img, (int(display_img.shape[1] / 2), 
                                                                     int(display_img.shape[0] / 2))))
                cv2.waitKey(1)
            
            
            if not is_empty:
                coco_data["images"].append({
                        "id": i,
                        "license": 1,
                        "width": display_img.shape[0],
                        "height": display_img.shape[1],
                        "file_name": img.split('/')[-1][:-4] + "_synth" + ".JPG"
                    }
                )
                cv2.imwrite(img_name, display_img)
                print("Saved", img_name)
            
        else:
            queueLock.release()
            
# setup as many threads as there are (virtual) CPUs
exitFlag_export = 0
# only use a fourth of the number of CPUs for export as hugin and enfuse utilise multi core processing in part
threadList_export = createThreadList(getThreads() * 4)
print("Using", len(threadList_export), "threads for export...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue_export = queue.Queue(len(dataset_img))
threads = []
threadID = 1

np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(len(colony['ID']), 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2

# we can additionally plot the points in the data files to check joint locations
# WARNING: At the moment there seems to be an issue with inccorrectly given joint locations
plot_joints = True

# remember to refine an export folder when saving out your dataset
generate_dataset = True

def fix_bounding_boxes(coords,max_val = [1024,1024]):
    # fix bounding box coordinates so they do not reach beyond the image
    fixed_coords = []
    for c, coord in enumerate(coords):
        if c == 0 or c == 2:
            max_val_temp = max_val[0]
        else:
            max_val_temp = max_val[1]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
# WARNING: At the moment the ID shown in segmentation maps does not always correspond to the ID in the data file (off by 1)
visibility_threshold = 0.015

timer = time.time()

# create output folder for used images
if not os.path.exists(target_dir + "/data"):
    os.mkdir(target_dir + "/data")

# Create new threads
for tName in threadList_export:
    thread = exportThread(threadID, tName, workQueue_export)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with stacks
queueLock.acquire()
for i,img in enumerate(dataset_img):
    workQueue_export.put([i, img])
queueLock.release()

# Wait for queue to empty
while not workQueue_export.empty():
    pass

# Notify threads it's time to exit
exitFlag_export = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main export Thread")

# close all windows if they were opened
cv2.destroyAllWindows()

print("Total time elapsed:",time.time()-timer,"seconds")

Using 48 threads for export...
Starting Thread_0
Starting Thread_1
Starting Thread_2
Starting Thread_3
Starting Thread_4
Starting Thread_5
Starting Thread_6
Starting Thread_7
Starting Thread_8
Starting Thread_9
Starting Thread_10Starting Thread_11

Starting Thread_12
Starting Thread_13
Starting Thread_14
Starting Thread_15
Starting Thread_16
Starting Thread_17
Starting Thread_18
Starting Thread_19
Starting Thread_20
Starting Thread_21
Starting Thread_22Starting Thread_23
Starting Thread_24

Starting Thread_25
Starting Thread_26
Starting Thread_27
Starting Thread_28
Starting Thread_29
Starting Thread_30
Starting Thread_31
Starting Thread_32
Starting Thread_33
Starting Thread_34Starting Thread_35

Starting Thread_36
Starting Thread_37
Starting Thread_38
Starting Thread_39
Starting Thread_40
Starting Thread_41
Starting Thread_42
Starting Thread_43Starting Thread_44

Starting Thread_45
Starting Thread_46
Starting Thread_47
Exiting Thread_30Exiting Thread_2

Exiting Thread_22
Exiting Thread

Now, dump it all into one sexy **COCO style json** file

In [8]:
with open(target_dir + '/labels.json', 'w', encoding='utf-8') as outfile:
    json.dump(coco_data, outfile, ensure_ascii=False, indent=4)